In [ ]:
# Common imports
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.utils import resample
# Where to save the figures and data files

def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    
    #adding a stochastic noise with a normal distribution between[0,0.1]
    
    noise = np.random.normal(0, 0.1, len(x)*len(x)) 
    noise = noise.reshape(len(x),len(x))
    return term1 + term2 + term3 + term4 + noise

def create_X(x, y, n ):
    if len(x.shape) > 1:
            x = np.ravel(x)   #gives a 1D array
            y = np.ravel(y)
    N = len(x)
    l = int((n+1)*(n+2)/2) # Number of elements in beta
    X = np.ones((N,l))
    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)
    return X

def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

def MSE(y_data, y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

N = 100
seed = 31415
np.random.seed(seed)
x = np.sort(np.random.uniform(0, 1, N)) #sort data to plot correctly
y = np.sort(np.random.uniform(0, 1, N))

x, y = np.meshgrid(x,y)  #x et y dim = 2 #Creates a grid for the evaluation of the 2 dimension Franke's function
z = FrankeFunction(x, y) #dim =2

#Creating 1D array of each coordinates 

x_dim1 = x.ravel() #dim =1
y_dim1 = y.ravel() #dim =1
z_dim1 = z.ravel() #dim =1



#Polynomial order
n = 10

#Set number of iterations
n_bootstraps=50

#Creating list to store the results 

mean_test_error=[]
mean_train_error=[]
mean_test_r2=[]
mean_train_r2=[]

bias = []
variance = []
bias_variance = []


for poly in range (1,n+1): #loop for the polynomial order
    
    testing_error=[]
    training_error =[]
    testing_r2=[]
    training_r2 =[]
    Z_predict=[]

    # Create the design matrix
    X = create_X(x_dim1,y_dim1, poly)

    #Splitting of the data

    X_train, X_test, z_train, z_test = train_test_split(X,z_dim1,test_size=0.2)
    
    scaler = StandardScaler()
    scaler.fit(X_train)

    
 # Begin iterations for the bootstrap method   
    for i in range(n_bootstraps):
        # Bootstrap resampling - leaving out 10% of the data
        X_resampled, z_ = resample(X_train, z_train,n_samples=int(z_train.shape[0]*0.9))
    
        #Standardize features by removing the mean and scaling to unit variance

        X_train_scaled = scaler.transform(X_resampled)
        X_test_scaled = scaler.transform(X_test)

        # OLS using scikit learn library
        clf = skl.LinearRegression().fit(X_train_scaled, z_)
        
        #Prediction of the z value using OLS model
        zpred = clf.predict(X) #dim = 1
        
        
        #Reshaping array without changing the data
        zplot = zpred.reshape(N, N) #dim =2
        
        #filling lists with the result for each bootstrap

        testing_error.append(MSE(z_test, clf.predict(X_test_scaled)))
        training_error.append(MSE(z_, clf.predict(X_train_scaled)))
        testing_r2.append(R2(z_test,clf.predict(X_test_scaled)))
        training_r2.append(R2(z_,clf.predict(X_train_scaled)))
        Z_predict.append(clf.predict(X_test_scaled))
        
    

    print('Polynomial degree:', poly)
    print("MSE: {:.2f}".format(MSE(z_test,clf.predict(X_test_scaled))))
    print("R2: {:.2f}".format(R2(z_test,clf.predict(X_test_scaled))))
    
    #Getting global results for each polynome degree

    variance.append(np.mean(np.var(Z_predict, axis=0, keepdims=True)))
    bias.append(np.mean( (z_test - np.mean(Z_predict, axis=0, keepdims=True))**2 ))
    bias_variance.append(variance[poly-1]+bias[poly-1])
    mean_test_error.append(np.mean(testing_error))
    mean_train_error.append(np.mean(training_error))
    mean_test_r2.append(np.mean(testing_r2))
    mean_train_r2.append(np.mean(training_r2))
    
    
print(bias)
print(variance)
print(bias_variance)


In [ ]:
#plot parameters to estimate quality of the fit
plt.figure(figsize=(6,4))
plt.plot(mean_train_error, label="MSE train")
plt.plot(mean_test_error, label="MSE test")
plt.legend(fontsize = 10)
plt.xticks([0,1,2,3,4,5,6,7,8,9],["1","2","3","4","5","6","7","8","9","10"], fontsize=12)
plt.xlabel("Polynomial degree", fontsize=14)
plt.ylabel("Error", fontsize = 14)
plt.yticks(fontsize=12)

plt.show()

In [ ]:

plt.figure(figsize=(6,4))
plt.plot(mean_train_r2, label=r'$r^2_{train}$')
plt.plot(mean_test_r2, label=r'$r^2_{test}$')
plt.legend(fontsize = 10)
plt.xticks([0,1,2,3,4,5,6,7,8,9],["1","2","3","4","5","6","7","8","9","10"], fontsize=12)
plt.xlabel("Polynomial degree", fontsize=14)
plt.ylabel(r'$r^2$', fontsize = 14)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(bias, label="Bias")
plt.plot(variance, label="Variance")
plt.legend(fontsize = 10)
plt.xticks([0,1,2,3,4,5,6,7,8,9],["1","2","3","4","5","6","7","8","9","10"], fontsize=12)
plt.xlabel("Polynomial degree", fontsize=14)
plt.ylabel("Error", fontsize = 14)
plt.yticks(fontsize=12)
plt.savefig("bias_variance_mse_ols_bootstrap_10deg.png", dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(bias_variance, label="bias+variance")
print(bias_variance)
plt.plot(mean_test_error, label="MSE test")
print(mean_test_error)
plt.legend(fontsize = 10)
plt.xticks([0,1,2,3,4,5,6,7,8,9],["1","2","3","4","5","6","7","8","9","10"], fontsize=12)
plt.xlabel("Polynomial degree", fontsize=14)
plt.ylabel("Error", fontsize = 14)
plt.yticks(fontsize=12)
plt.savefig("bias_variance_error_mse_ols_bootstrap_10deg.png", dpi=300)
plt.show()

In [ ]:
#plot result fit
fig = plt.figure(figsize=(20,8))

ax = plt.axes(projection='3d')
ax.set_zlim(-0.10, 0.7)
ax.plot_surface(x,y,zplot,cmap=cm.coolwarm,linewidth=0,antialiased=False)
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('y', fontsize=18)
ax.set_zlabel('z', fontsize=18)
plt.show()

In [ ]:
#plot franke function
fig = plt.figure(figsize=(20,8))

ax = plt.axes(projection='3d')
ax.set_zlim(-0.10, 1.40)
ax.plot_surface(x,y,z,cmap=cm.coolwarm,linewidth=0,antialiased=False)
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('y', fontsize=18)
ax.set_zlabel('z', fontsize=18)
plt.show()